In [95]:
from sklearn.datasets import load_iris
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import pairwise_distances

import cProfile


def getPermutation(totalRange,numberElements):
    random_seed = 10312003
    rng = np.random.RandomState(random_seed)
    permutation = rng.permutation(totalRange)
    return permutation[:numberElements]


def onlineKmeans(X,k=3,b=30,maxiter=1000):
    centroids = X[getPermutation(len(X),k)]
    pointsPerClusters = np.zeros([k,1])
    for i in range(maxiter):
        M=X[getPermutation(len(X),b)]
        distances = pairwise_distances(M, centroids, metric='euclidean')
        nearestCenters = np.argmin(distances, axis=1)
        for iter, x in enumerate(M):
            centerIndex = nearestCenters[iter]
            pointsPerClusters[centerIndex] = pointsPerClusters[centerIndex] + 1
            eta = 1/pointsPerClusters[centerIndex]
            centroids[centerIndex] = (1 - eta)*centroids[centerIndex] + eta * x

    return centroids

In [102]:
from timeit import Timer
from functools import partial
from memory_profiler import memory_usage


def get_execution_time(function, numberOfExecTime, *args, **kwargs):
    """Return the execution time of a function in seconds."""
    time =round(Timer(partial(function, *args, **kwargs)).timeit(numberOfExecTime), 5)
    return time

def wrapOnlineKmeans(X,k=3,b=30,maxiter=1000):
    return get_execution_time(onlineKmeans,1,X,k,b,maxiter)
    

In [104]:
def getExTimeAndMemoryUsage(function, *args, **kwargs):
    """Return the execution time of a function in seconds and the memory usage"""
    #executionTime=round(Timer(partial(function, *args, **kwargs))
    #             .timeit(1), 5)
    
    memoryUsage=memory_usage((test), max_usage=True)[0]
    return executionTime,memoryUsage 

iris = load_iris()
X=iris.data
random_seed = 10312003
rng = np.random.RandomState(random_seed)
permutation = rng.permutation(len(X))
X = X[permutation]
memory, time = memory_usage((wrapOnlineKmeans, (X,), {'k':int(6),'b':int(33)}),max_usage=True, retval=True)
print memory[0], time

69.3984375 0.45492
